<a href="https://colab.research.google.com/github/NehaKumari500092077/Machine-Learning-Lab/blob/main/Session_9_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Assignment 9**: Random Forest Regression

Objective: This assignment provides a hands-on understanding of the Random Forest Regression algorithm. You will implement the algorithm from scratch and compare its performance with the scikit-learn implementation. You will also tune hyperparameters using cross-validation.

Dataset:  Diabetes dataset (https://scikit-learn.org/1.5/modules/generated/sklearn.datasets.load_diabetes.html#sklearn.datasets.load_diabetes)

Tasks:








1. Data Preparation: [1 Marks]
*   Load the diabetes dataset using sklearn.datasets.load_diabetes.
*   Split the dataset into training and test sets with a 80%-20% ratio.

In [1]:
from sklearn.datasets import load_diabetes
import pandas as pd
import numpy as np


diabetes_data = load_diabetes()
X = diabetes_data.data
Y = diabetes_data.target
dataset = pd.DataFrame(data=np.c_[X, Y], columns=diabetes_data.feature_names + ['target'])
print(dataset.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 442 entries, 0 to 441
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   age     442 non-null    float64
 1   sex     442 non-null    float64
 2   bmi     442 non-null    float64
 3   bp      442 non-null    float64
 4   s1      442 non-null    float64
 5   s2      442 non-null    float64
 6   s3      442 non-null    float64
 7   s4      442 non-null    float64
 8   s5      442 non-null    float64
 9   s6      442 non-null    float64
 10  target  442 non-null    float64
dtypes: float64(11)
memory usage: 38.1 KB
None


In [2]:
print(dataset.head())

        age       sex       bmi        bp        s1        s2        s3  \
0  0.038076  0.050680  0.061696  0.021872 -0.044223 -0.034821 -0.043401   
1 -0.001882 -0.044642 -0.051474 -0.026328 -0.008449 -0.019163  0.074412   
2  0.085299  0.050680  0.044451 -0.005670 -0.045599 -0.034194 -0.032356   
3 -0.089063 -0.044642 -0.011595 -0.036656  0.012191  0.024991 -0.036038   
4  0.005383 -0.044642 -0.036385  0.021872  0.003935  0.015596  0.008142   

         s4        s5        s6  target  
0 -0.002592  0.019907 -0.017646   151.0  
1 -0.039493 -0.068332 -0.092204    75.0  
2 -0.002592  0.002861 -0.025930   141.0  
3  0.034309  0.022688 -0.009362   206.0  
4 -0.002592 -0.031988 -0.046641   135.0  


In [3]:
# check for duplicates
duplicates_value = dataset.duplicated().sum()
print(f'No. of Duplicates: {duplicates_value}')

No. of Duplicates: 0


In [4]:
# check for missing values
missing_value = dataset.isnull().sum()
print(f'Total Number of Missing Values: {missing_value.sum()}')

Total Number of Missing Values: 0


In [5]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
dataset = sc.fit_transform(dataset)

# convert scaled data back to dataframe
dataset = pd.DataFrame(dataset, columns=diabetes_data.feature_names + ['target'])

In [12]:
# Split dataset into traing and test set
from sklearn.model_selection import train_test_split

X = dataset.drop(columns='target')
Y = dataset['target']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20, random_state=42)

# Convert to NumPy arrays
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()
Y_train = Y_train.to_numpy()
Y_test = Y_test.to_numpy()


2. From-Scratch Implementation: [7 Marks]
* Implement the Random Forest Regression algorithm from scratch in Python.

In [16]:
class DecisionTreeRegression:

  def __init__(self, max_depth=5, max_features='sqrt'):
    self.max_depth = max_depth # Maximum depth of the tree
    self.max_features = max_features # Number of features to consider for each split




  def fit(self, X, Y):
    self.no_of_features = X.shape[1] # Number of features in the dataset
    if(self.max_features == 'sqrt'):
      self.max_features = int(np.sqrt(self.no_of_features)) # Calculate the number of features to use
    self.tree_ = self._build_tree(X, Y)  # Build the tree




  def _build_tree(self, X, Y, depth = 0):
    no_of_samples, no_of_features = X.shape # Get the number of samples and features
    if(depth >= self.max_depth or no_of_samples < 2): # Stopping criteria: Maximum depth reached, Not enough samples to split
      return np.mean(Y) # Return the mean of the target values

    # Select random subset of features
    feature_indices = np.random.choice(no_of_features, size=self.max_features, replace=False)

    # Select best split from random subset
    best_mean_squared_error = float('inf')
    best_feature_index = None
    best_threshold = None

    for index in feature_indices: # Iterate through selected feature
      sorted_indices = np.argsort(X[:, index])  # Get indices that would sort the feature values
      thresholds = X[sorted_indices, index]  # Sort the thresholds
      values = Y[sorted_indices]  # Sort the corresponding target values

      for i in range(1, no_of_samples):
         if(values[i - 1] != values[i]): # Check if the values changes between consecutive samples
          thr = (thresholds[i - 1] + thresholds[i]) / 2
          left = [Y[j] for j in range(no_of_samples) if X[j, index] < thr] # Samples in the left child
          right = [Y[j] for j in range(no_of_samples) if X[j, index] >= thr] # Samples in the right child
          # Handle empty or single-element arrays
          var_left = np.var(left) if len(left) > 1 else 0
          var_right = np.var(right) if len(right) > 1 else 0

          # Check for zero division
          if no_of_samples > 0:
            mean_square_error = ((len(left) * var_left) + (len(right) * var_right)) / no_of_samples
          #else:
           # mean_square_error = 0


          if mean_square_error < best_mean_squared_error: # Check if this split is better
            best_mean_squared_error = mean_square_error
            best_feature_index = index
            best_threshold = thr

    if(best_mean_squared_error == float('inf')): # If no split found
      return np.mean(Y) # Return mean as leaf value

    # Recur to build the left and right subtrees
    left_indexes = np.where(X[:, best_feature_index] < best_threshold) # Indices of samples in the left child
    right_indexes = np.where(X[:, best_feature_index] >= best_threshold) # Indices of samples in the right child
    left = self._build_tree(X[left_indexes], Y[left_indexes], depth + 1) # Recursively build left subtree
    right = self._build_tree(X[right_indexes], Y[right_indexes], depth + 1) # Recursively build right subtree
    return (best_feature_index, best_threshold, left, right) # Return the node (feature index, threshold, left child, right child)



  def predict(self, X): # predicts the class labels for a given feature matrix X
        return [self._predict(inputs) for inputs in X] # Predict for each sample in X



  def _predict(self, inputs): # traverses the tree to make a prediction for a single sample.
    node = self.tree_ # Start at the root node
    while isinstance(node, tuple):
      index, thr, left, right = node
      if inputs[index] < thr:
        node = left
      else:
        node = right
    return node  # Return the mean value at the leaf node



class RandomForestRegression:

  def __init__(self, no_of_trees=100, max_depth=5, max_features='sqrt'):
    self.no_of_trees = no_of_trees # Number of trees in the forest
    self.max_depth = max_depth # Maximum depth of each tree
    self.max_features = max_features # Number of features to consider for each split

  def fit(self, X, Y):
    self.trees = []  # List to store the decision trees
    self.indices = [] # List to store the indices used for bootstrapping
    no_of_samples = X.shape[0] # Number of samples
    for _ in range(self.no_of_trees): # Iterate to create no_of_trees
      tree = DecisionTreeRegression(max_depth=self.max_depth, max_features=self.max_features) # Create a DecisionTree
      indices = np.random.choice(no_of_samples, size=no_of_samples, replace=True) # Bootstrap samples
      tree.fit(X[indices], Y[indices]) # Train the tree on bootstrapped samples; For each tree, it generates a bootstrap sample (random sampling with replacement)
      self.trees.append(tree) # Add the trained tree to the list
      self.indices.append(indices) # Store the indices used for bootstrapping

  def predict(self, X):
    # Get predictions from each tree and return their mean
    predictions = np.array([tree.predict(X) for tree in self.trees]) # Get predictions from each tree
    return np.mean(predictions, axis=0) # Return the mean of the predictions



def calculate_oob_error(X, y, forest):
    no_of_samples = X.shape[0] # Number of samples
    no_of_trees = forest.no_of_trees # Number of trees
    predictions = np.zeros((no_of_samples, no_of_trees)) # Array to store OOB predictions


    for i, tree in enumerate(forest.trees): # Iterate over trees
        indices = forest.indices[i] # Indices used to train the current tree
        oob_indices = np.array([index for index in range(no_of_samples) if index not in indices]) # Out-of-bag indices
        if len(oob_indices) > 0: # If there are OOB samples
          predictions[oob_indices, i] = tree.predict(X[oob_indices]) # Predict for OOB samples


    # Calculate the mean prediction for each sample
    oob_predictions = np.mean(predictions, axis=1) # Average predictions along the tree axis

    # Calculate Mean Squared Error (MSE) only for samples with predictions
    valid_indices = ~np.isnan(oob_predictions)  # Find indices where predictions are valid
    oob_err = np.mean((y[valid_indices] - oob_predictions[valid_indices]) ** 2) if np.any(valid_indices) else None

    return oob_err  # Return the mean squared error as the OOB error


In [17]:
# Create and train Random Forest with feature randomness
rf = RandomForestRegression(no_of_trees=100, max_depth=5, max_features='sqrt')
rf.fit(X_train, Y_train)

# Calculate out-of-bag error
oob_err = calculate_oob_error(X_train, Y_train, rf)
print("Out-of-Bag Error:", oob_err)

# Predict on the test set
Y_pred = rf.predict(X_test)

# Calculate regression metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

mean_squared_error = mean_squared_error(Y_test, Y_pred)  # Mean Squared Error
print("Mean Squared Error by scratch implementation:", mean_squared_error)


Out-of-Bag Error: 0.7632908033959095
Mean Squared Error by scratch implementation: 0.47857137332314037


In [18]:
import matplotlib.pyplot as plt

# Define a function to plot a decision tree in a regression context
def plot_regression_tree(tree, feature_names, depth=0):
    """
    Recursively prints the structure of the decision tree, adapted for regression.
    Displays predicted values at leaf nodes and splits at decision nodes.
    """
    if isinstance(tree, tuple):
        idx, thr, left, right = tree
        feature_name = feature_names[idx]
        # Print the condition at the decision node
        print('  ' * depth + f'if {feature_name} <= {thr:.2f}:')  # Display the threshold value with 2 decimal precision
        plot_regression_tree(left, feature_names, depth + 1)      # Recursive call for left subtree
        print('  ' * depth + f'else:')
        plot_regression_tree(right, feature_names, depth + 1)     # Recursive call for right subtree
    else:
        # Leaf node: display the predicted average value for regression
        print('  ' * depth + f'Predicted value: {tree:.2f}')       # Display the prediction with 2 decimal precision

# Create a list of feature names for visualization
feature_names = [f'Feature {i}' for i in range(X_train.shape[1])]

# Select a few decision trees for visualization
trees_to_visualize = [0, 1, 2, 3, 4]  # Adjust the number of trees you want to visualize

# Visualize each selected decision tree
for i in trees_to_visualize:
    print(f"\nDecision Tree {i+1}:")
    plot_regression_tree(rf.trees[i].tree_, feature_names)
    print()



Decision Tree 1:
if Feature 8 <= -0.02:
  if Feature 8 <= -0.91:
    if Feature 6 <= 1.06:
      if Feature 8 <= -1.10:
        if Feature 7 <= -0.02:
          Predicted value: -0.71
        else:
          Predicted value: 0.14
      else:
        if Feature 6 <= 0.56:
          Predicted value: -1.31
        else:
          Predicted value: -0.81
    else:
      if Feature 0 <= -1.30:
        Predicted value: -1.18
      else:
        if Feature 0 <= 0.69:
          Predicted value: -1.35
        else:
          Predicted value: -1.47
  else:
    if Feature 2 <= 0.15:
      if Feature 4 <= -1.23:
        if Feature 8 <= -0.42:
          Predicted value: 0.14
        else:
          Predicted value: -0.77
      else:
        if Feature 5 <= 1.78:
          Predicted value: -0.81
        else:
          Predicted value: 1.31
    else:
      if Feature 4 <= -1.07:
        Predicted value: 0.75
      else:
        if Feature 8 <= -0.56:
          Predicted value: 0.95
        else:
   

3. Hyperparameter Tuning (From-Scratch Implementation): [5 Marks]
* Use GridSearchCV from scikit-learn to find the optimal number of trees for your from-scratch implementation.
* Evaluate the model with the best number of trees on the test set and report the Mean Squared Error (MSE) and the OOB error.


5. Scikit-learn Implementation: [5 Marks]
* Use RandomForestRegressor from scikit-learn to train a model on the diabetes dataset.
* Using cross-validation find the optimal number of trees for the scikit-learn implementation.
* Evaluate the model with the best number of trees on the test set and report the MSE and OOB.



6. Comparison and Visualization: [2 Marks]
* Compare the performance (MSE) of your from-scratch implementation with the scikit-learn implementation.
* Create scatter plots to visualize:
  * Predicted values vs. true values for your from-scratch implementation.
  * Predicted values vs. true values for the scikit-learn implementation.
* Display these plots side-by-side for easy comparison.